<a href="https://colab.research.google.com/github/codebyshennan/nyp_applied_ai/blob/main/ITI102-DSFoundation/assignment2/ITI102_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ITI102 Assignment 2

Name:   Wong Shen Nan

Admin no:    23A070B

## Q1
a) Use the python Scrapy module to implement a web scraping program to
get the content from the following websites (4 marks)

In [82]:
!pip -q install scrapy crochet folium scipy scikit-learn nltk

In [83]:
# Crochet provides the following basic APIs:
# Allow blocking code to call into Twisted and block until results are available or a timeout is hit, using the crochet.wait_for decorator.
# A lower-level API (crochet.run_in_reactor) allows blocking code to run code “in the background” in the Twisted thread, with the ability to repeatedly check if it’s done.
from crochet import setup, wait_for
setup()

In [84]:
import scrapy
from scrapy.crawler import CrawlerRunner
import json
from urllib.parse import urlparse

base_url = 'https://webscraper.io/test-sites/e-commerce/allinone/computers/'
slugs = ['tablets', 'laptops']

class ProductsToJson(scrapy.Spider):
    name = "WebScrapProductsToJson"
    start_urls = [ base_url + slug for slug in slugs]
    custom_settings = {
        'FEEDS': {
            'products.json': {
                'format': 'json',
                'overwrite': True
            }
        }
    }

    """
    requirement:
    - {
        “type”: “tablet”,
        "price": ["$603.99"],
        "description": ["Wi-Fi, 64GB, Silver"],
        "product": "Apple iPad Air",
        "review": "7 reviews"
      }
    """
    def parse(self, response):
        """parse data from urls"""
        parsed_url = urlparse(response.request.url)
        slug = parsed_url.path.strip('/').split('/')[-1]

        for meta in response.css('body > div.wrapper > div.container.test-site > div.row > div.col-md-9 > div.row > div > div.thumbnail'):
          price = meta.css('div.thumbnail > div.caption > h4::text')[0].get()
          product = meta.css('div.thumbnail > div.caption > h4 > a::text')[0].get()
          description = meta.css('div.thumbnail > div.caption > p.description::text').get()
          ratings = meta.css('div.thumbnail > div.ratings > p.pull-right::text').get()

          thumbnail = {
              "type": slug,
              "price": [price],
              "description": [description],
              "product": product,
              "review": ratings
          }

          yield thumbnail

@wait_for(10)
def run_spider():
    """run spider with WebScrapeProductsToJson"""
    crawler = CrawlerRunner()
    d = crawler.crawl(ProductsToJson)
    return d

In [85]:
run_spider()


It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.


In [86]:
import pandas as pd
import json

with open('./products.json', 'r') as file:
    data = json.load(file)

df = pd.DataFrame(data)

df

,type,price,description,product,review
0,tablets,[$69.99],"[7"" screen, Android]",Lenovo IdeaTab,7 reviews
1,tablets,[$88.99],"[Black, 7"" IPS, Quad-Core 1.2GHz, 8GB, Android...",IdeaTab A3500L,7 reviews
2,tablets,[$96.99],"[7"" screen, Android, 16GB]",Acer Iconia,7 reviews
3,tablets,[$97.99],"[7"", 8GB, Wi-Fi, Android 4.2, White]",Galaxy Tab 3,2 reviews
4,tablets,[$99.99],"[Black, 7"", 1.6GHz Dual-Core, 8GB, Android 4.4]",Iconia B1-730HD,1 reviews
...,...,...,...,...,...
133,laptops,[$1399.00],"[Lenovo Legion Y720, 15.6"" FHD IPS, Core i7-77...",Lenovo Legion Y7...,8 reviews
134,laptops,[$1399.00],"[Asus ROG Strix GL702VM-GC146T, 17.3"" FHD, Cor...",Asus ROG Strix G...,10 reviews
135,laptops,[$1769.00],"[Asus ROG Strix GL702ZC-GC154T, 17.3"" FHD, Ryz...",Asus ROG Strix G...,7 reviews
136,laptops,[$1769.00],"[Asus ROG Strix GL702ZC-GC209T, 17.3"" FHD IPS,...",Asus ROG Strix G...,8 reviews


b) Develop a python function to search for a product information based on the
review. (2 marks)

In [87]:
def search_product(filepath, query, key):
  with open(filepath) as file:
    data = json.load(file)

  results = [item for item in data if key in item and query in item[key]]
  return results

In [88]:
print("There are", len(search_product('./products.json', '8', 'review')), "products with 8 reviews.")

There are 12 products with 8 reviews.


In [89]:
print("There are", len(search_product('./products.json', '14', 'review')), "products with 14 reviews.")

There are 11 products with 14 reviews.


## Q2:
Implement a Singapore traffic report system using python.

a) Python program request for the traffic incident using URL and get the return JSON data. Extract the traffic incident data and display in the Singapore map (3 marks)

ref: https://georgetsilva.github.io/posts/mapping-points-with-folium/

In [90]:
import requests

url = "http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents"
headers = {
    'AccountKey': '8JUcVpAQQqiLFJMkisjndw==',
    # 'X-API-Key': 'token'
}
payload = {}

response = requests.request("GET",
                            url,
                            headers = headers,
                            data = payload
                            )

trf_data = response.json()
df_incidents = pd.json_normalize(trf_data, record_path = ['value'])
# print(json.dumps(trf_data, indent=2))
df_incidents

,Type,Latitude,Longitude,Message
0,Roadwork,1.340466,103.805068,(2/7)22:53 Roadworks on PIE (towards Changi Ai...
1,Roadwork,1.323437,103.718801,(2/7)22:45 Roadworks on AYE (towards MCE) befo...
2,Roadwork,1.321932,103.704705,(2/7)22:33 Roadworks on AYE (towards MCE) afte...
3,Roadwork,1.331494,103.956715,(2/7)22:29 Roadworks on Xilin Avenue (towards ...
4,Roadwork,1.388324,103.743791,(2/7)22:28 Roadworks on KJE (towards PIE) afte...
5,Roadwork,1.332860,103.955429,(2/7)22:27 Roadworks on Upper Changi Road East...
6,Roadwork,1.366584,103.710002,(2/7)22:22 Roadworks on KJE (towards PIE) befo...
7,Roadwork,1.331231,103.976195,(2/7)21:43 Roadworks on ECP (towards Changi Ai...
8,Heavy Traffic,1.329891,103.863425,(2/7)20:42 Heavy Traffic on PIE (towards Tuas)...
9,Roadwork,1.321808,103.708748,(2/7)20:25 Roadworks on AYE (towards MCE) at J...


In [91]:
locations = df_incidents[['Latitude', 'Longitude']]
locList = locations.values.tolist()

In [92]:
import folium
from folium.plugins import MarkerCluster

map = folium.Map(location=[1.29, 103.85], zoom_start=11, tiles = "cartodb positron")

marker_cluster = MarkerCluster().add_to(map)

for point in range(0, len(locList)):
    folium.Marker(locList[point], popup=df_incidents['Message'][point]).add_to(marker_cluster)
map

b) Python program request for the traffic band using URL and get the return JSON data. Extract the traffic band data and display in the Singapore map (3 marks)

In [93]:
url = "http://datamall2.mytransport.sg/ltaodataservice/TrafficSpeedBandsv2"
headers = {
    'AccountKey': '8JUcVpAQQqiLFJMkisjndw==',
    # 'X-API-Key': 'token'
}
payload = {}

response = requests.request("GET",
                            url,
                            headers = headers,
                            data = payload
                            )

trf_data = response.json()
df_bands = pd.json_normalize(trf_data, record_path = ['value'])
# print(json.dumps(trf_data, indent=2))
df_bands

,LinkID,RoadName,RoadCategory,SpeedBand,MinimumSpeed,MaximumSpeed,Location
0,103000000,KENT ROAD,E,4,30,39,1.3170142376560023 103.85298052044503 1.316684...
1,103000010,BUCKLEY ROAD,E,7,60,69,1.3166507852203482 103.84102305136321 1.316912...
2,103000011,BUCKLEY ROAD,E,7,60,69,1.316912438354752 103.84022564204443 1.3166507...
3,103000014,SHREWSBURY ROAD,E,7,60,69,1.3186726294030418 103.84700267615683 1.318021...
4,103000015,SHREWSBURY ROAD,E,8,70,999,1.3180212058893457 103.84711392225329 1.318672...
...,...,...,...,...,...,...,...
495,103001218,DRAYCOTT DRIVE,D,8,70,999,1.3096279339357806 103.83110989321554 1.309694...
496,103001219,DRAYCOTT DRIVE,D,5,40,49,1.309694857000047 103.83119447352243 1.3096279...
497,103001226,PADANG JERINGAU,D,5,40,49,1.3099051585628567 103.86554520014256 1.309253...
498,103001227,PADANG JERINGAU,D,6,50,59,1.309253800591554 103.86611733337693 1.3099051...


In [94]:
# df_bands['RoadCategory'].unique()
df_bands['SpeedBand'].unique()

array([4, 7, 8, 2, 5, 6, 3, 1])

In [95]:
colors = ["red", "green", "blue", "yellow", "orange", "purple", "pink", "black"]

def regioncolors(row):
    return colors[int(row['SpeedBand'])-1]
df_bands["color"] = df_bands.apply(regioncolors, axis=1)
df_bands

,LinkID,RoadName,RoadCategory,SpeedBand,MinimumSpeed,MaximumSpeed,Location,color
0,103000000,KENT ROAD,E,4,30,39,1.3170142376560023 103.85298052044503 1.316684...,yellow
1,103000010,BUCKLEY ROAD,E,7,60,69,1.3166507852203482 103.84102305136321 1.316912...,pink
2,103000011,BUCKLEY ROAD,E,7,60,69,1.316912438354752 103.84022564204443 1.3166507...,pink
3,103000014,SHREWSBURY ROAD,E,7,60,69,1.3186726294030418 103.84700267615683 1.318021...,pink
4,103000015,SHREWSBURY ROAD,E,8,70,999,1.3180212058893457 103.84711392225329 1.318672...,black
...,...,...,...,...,...,...,...,...
495,103001218,DRAYCOTT DRIVE,D,8,70,999,1.3096279339357806 103.83110989321554 1.309694...,black
496,103001219,DRAYCOTT DRIVE,D,5,40,49,1.309694857000047 103.83119447352243 1.3096279...,orange
497,103001226,PADANG JERINGAU,D,5,40,49,1.3099051585628567 103.86554520014256 1.309253...,orange
498,103001227,PADANG JERINGAU,D,6,50,59,1.309253800591554 103.86611733337693 1.3099051...,purple


In [96]:
# Empty list to store parsed values
# startlon, startlat, endlon, endlat



split_values = [location_str.split() for location_str in df_bands["Location"]]
start_lat, start_long, end_lat, end_long = zip(*[(float(s[0]), float(s[1]), float(s[2]), float(s[3])) for s in split_values])


latlong_origin = list(zip(start_lat, start_long))
latlong_destination = list(zip(end_lat, end_long))

c) Use different markers to represent and differentiate the traffic incident and traffic bands in the map.Allow the markers to display more information when it require (3 marks)

In [97]:
for index, (origin, destination) in enumerate(zip(latlong_origin, latlong_destination)):
    folium.CircleMarker([origin[0], origin[1]],
                        radius=15,
                        fill_color="#3db7e4", # divvy color
                       ).add_to(marker_cluster)

    folium.CircleMarker([destination[0], destination[1]],
                        radius=15,
                        fill_color="red", # divvy color
                       ).add_to(marker_cluster)

    folium.PolyLine([[origin[0], origin[1]],
                     [destination[0], destination[1]]],
                    color=df_bands['color'][index],
                    popup= 'Road: ' + df_bands['RoadName'][index] + ' (' + df_bands['RoadCategory'][index] + str(df_bands['SpeedBand'][index]) + ') \n\n\n' + 'Max Speed: ' + df_bands['MaximumSpeed'][index] + 'km/h' + '\n\n\n' + 'Min Speed: ' + df_bands['MinimumSpeed'][index] + 'km/h'
                    ).add_to(marker_cluster)

map

## Q3

You are given a set of text data that expresses the sentiments of customers.

The sentiments are label as follow:
```
pos- positive
neg- negative
```

The text data are stored in the Train and Test folders with two subfolders `pos` and `neg`. In each of these, the subfolder contains 100 text reviews.

Refer to the `Q3sentimentClassificationV2_student.ipynb`.

Complete the data preprocessing tasks in the ipynb file using `python` Natural Language Toolkit( https://www.nltk.org/).

## Data preparation

In [98]:
from google.colab import drive
drive.mount('/content/drive')
data_dir_path='/content/drive/My Drive/Data/DSA2/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [99]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [100]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

In [101]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

###Step 1
use the 100 text files in dataset/train/pos/ dir format to dataframe as follow:   
   
```
       Reviews                 |          labels
-------------------------------------------------------
text from file 1               |            1
text from file 2               |            1
text from file 3               |            1
......                         |            1
......                         |            1
```



In [102]:
path = data_dir_path+"dataset/train/pos/"
temp = []
for file in os.listdir(path):
    with open(os.path.join(path + file), "r") as f:
        temp.append(f.readlines()[0])

first_1000 = temp[:100]
print(first_1000)
pos = pd.DataFrame({"reviews": first_1000, "labels": list(np.ones(len(first_1000), dtype=int))})
pos.head()
pos.info()

['Bromwell High is nothing short of brilliant. Expertly scripted and perfectly delivered, this searing parody of a students and teachers at a South London Public School leaves you literally rolling with laughter. It\'s vulgar, provocative, witty and sharp. The characters are a superbly caricatured cross section of British society (or to be more accurate, of any society). Following the escapades of Keisha, Latrina and Natella, our three "protagonists" for want of a better term, the show doesn\'t shy away from parodying every imaginable subject. Political correctness flies out the window in every episode. If you enjoy shows that aren\'t afraid to poke fun of every taboo subject imaginable, then Bromwell High will not disappoint!', "'Stanley and Iris' show the triumph of the human spirit. For Stanley, it's the struggle to become literate and realize his potential. For Iris, it's to find the courage to love again after becoming a widow. The beauty of the movie is the dance that Robert DeNi

###Step 2
use the 100 text files in dataset/train/neg/ dir format to dataframe as follow:   
   
```
       Reviews                 |          labels
-------------------------------------------------------
text from file 1               |            0
text from file 2               |            0
text from file 3               |            0
......                         |            0
......                         |            0
```

In [103]:
path =  data_dir_path+"dataset/train/neg/"
temp = []
for file in os.listdir(path):
    with open(os.path.join(path + file), "r") as f:
        temp.append(f.readlines()[0])

first_1000 = temp[:100]
neg = pd.DataFrame({"reviews": first_1000, "labels": list(np.zeros(len(first_1000), dtype=int))})

###Step 3
Combine the 2 dataframe in Step1 and Step2  as follow:   
   
```
       Reviews                 |          labels
-------------------------------------------------------
text from file 1               |            1
text from file 2               |            1
text from file 3               |            1
......                         |            ..
......                         |            ..
text from file n               |            0
```

In [104]:
train_data = pd.concat([pos, neg], ignore_index=True)
# train_data = pd.concat([pos, neg])
print(train_data.head())
train_data.info()

                                             reviews  labels
0  Bromwell High is nothing short of brilliant. E...       1
1  'Stanley and Iris' show the triumph of the hum...       1
2  "All the world's a stage and its people actors...       1
3  If you fast forward through the horrible singi...       1
4  I found this to be a so-so romance/drama that ...       1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   reviews  200 non-null    object
 1   labels   200 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.2+ KB


###Step 4
use the 100 text files in dataset/test/pos/ dir format to dataframe as follow:   
   
```
       Reviews                 |          labels
-------------------------------------------------------
text from file 1               |            1
text from file 2               |            1
text from file 3               |            1
......                         |            1
......                         |            1
```

In [105]:
path = data_dir_path+"dataset/test/pos/"
temp = []
for file in os.listdir(path):
    with open(os.path.join(path + file), "r") as f:
        temp.append(f.readlines()[0])

first_1000 = temp[:100]
pos = pd.DataFrame({"reviews": first_1000, "labels": list(np.ones(len(first_1000), dtype=int))})

###Step 5
use the 100 text files in dataset/test/neg/ dir format to dataframe as follow:   
   
```
       Reviews                 |          labels
-------------------------------------------------------
text from file 1               |            0
text from file 2               |            0
text from file 3               |            0
......                         |            0
......                         |            0
```

In [106]:
path = data_dir_path+"dataset/test/neg/"
temp = []
for file in os.listdir(path):
    with open(os.path.join(path + file), "r") as f:
        temp.append(f.readlines()[0])

first_1000 = temp[:100]
neg = pd.DataFrame({"reviews": first_1000, "labels": list(np.zeros(len(first_1000), dtype=int))})

###Step 6
Combine the 2 dataframe in Step5 and Step6  as follow:   
   
```
       Reviews                 |          labels
-------------------------------------------------------
text from file 1               |            1
text from file 2               |            1
text from file 3               |            1
......                         |            ..
......                         |            ..
text from file n               |            0
```

In [107]:
test_data = pd.concat([pos, neg], ignore_index=True)
# test_data = pd.concat([pos, neg])
print(test_data.head)

<bound method NDFrame.head of                                                reviews  labels
0    I went and saw this movie last night after bei...       1
1    This movie was sadly under-promoted but proved...       1
2    I work at a movie theater and every Thursday n...       1
3    I was pleasantly surprised to find this movie ...       1
4    How many movies are there that you can think o...       1
..                                                 ...     ...
195  Okay, so it was never going to change the worl...       0
196  I'm the type of guy who loves hood movies from...       0
197  <br /><br />Never ever take a film just for it...       0
198  Beware, My Lovely (1952) Dir: Harry Horner <br...       0
199  Four things intrigued me as to this film - fir...       0

[200 rows x 2 columns]>


In [108]:
train_data.labels.value_counts()

1    100
0    100
Name: labels, dtype: int64

##Task 1 Tokenization ( 2 marks)

Use the train_data and test_data to continue the rest of the tasks.<br>

TODO:

Use Tokenizer to perform tokenize on the train_data and test_data "reviews" column.<br>





Complete the tokenization and store the result back into the train_data and test_data "reviews" column.


In [109]:
train_data['tokenized_reviews'] = train_data['reviews'].apply(word_tokenize)
test_data['tokenized_reviews'] = test_data['reviews'].apply(word_tokenize)

##Task 2 Remove Stop words (2 marks)

TODO:

Use the stop word in nltk


```
from nltk.corpus import stopwords  
stop_words = set(stopwords.words('english'))
```

Based the set of stop_words remove the stop words in the train and test dataset "reviews" column.<br>
Complete the stop_words removal and store the result back into the train_data and test_data "reviews" column.

In [110]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [111]:
train_data['stopwords_removed'] = train_data['tokenized_reviews'].apply(lambda x: [word for word in x if word.lower() not in stop_words])
test_data['stopwords_removed'] = test_data['tokenized_reviews'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

##Task 3 Lemmatization  ( 2 marks)
   
TODO:

Resolving words to their dictionary form    

  
In the lemmatization process resolve words to their dictionary form in the train and test dataset "reviews" column.<br>
Complete the lemmatization and store the result back into the train_data and test_data "reviews" column.



In [112]:
lemmatizer = WordNetLemmatizer()

train_data['lemmatized_reviews'] = train_data['stopwords_removed'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
test_data['lemmatized_reviews'] = test_data['stopwords_removed'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

##Task 4 Remove unwanted symbols ( 1 mark)
TODO:

Given  garbage = "~`!@#$%^&*()_-+={[}]|\:;'<,>.?/"  
  
Remove the unwanted symbols in the train and test dataset "reviews" column.<br>
Complete the unwanted symbols removal and store the result back into the train_data and test_data "reviews" column.


In [113]:
garbage = "~`!@#$%^&*()_-+={[}]|:;'<,>.?/"
unwanted_symbols = list(garbage)

train_data['cleaned_reviews'] = train_data['lemmatized_reviews'].apply(lambda x: [word for word in x if word not in unwanted_symbols])
test_data['cleaned_reviews'] = test_data['lemmatized_reviews'].apply(lambda x: [word for word in x if word not in unwanted_symbols])

##Task 5 Text Feature Extraction with TFIDF  ( 2 marks)

TODO:

This task is to do the text feature extraction with TFIDF for the train and test dataset "reviews" column.      

The text feature extraction with TFIDF are stored in dataframe:

train_features

test_features



In [114]:
tfidf_vectorizer = TfidfVectorizer()

train_features = tfidf_vectorizer.fit_transform(train_data['cleaned_reviews'].apply(lambda x: ' '.join(x))).toarray()
test_features = tfidf_vectorizer.transform(test_data['cleaned_reviews'].apply(lambda x: ' '.join(x))).toarray()

train_labels = train_data["labels"]
test_labels = test_data["labels"]

In [115]:
train_df = pd.DataFrame(train_features, columns=tfidf_vectorizer.get_feature_names_out())
test_df = pd.DataFrame(test_features, columns=tfidf_vectorizer.get_feature_names_out())

In [116]:
train_df

,00,000,08,10,100,11,120,14,15,16,...,yuppie,zack,zandt,zantara,zappa,zero,zombie,zombies,zoom,zsigmond
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
196,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
197,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
198,0.0,0.055571,0.0,0.0,0.0,0.0,0.0,0.0,0.050988,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.047737,0.063404,0.0,0.0


In [117]:
train_labels

0      1
1      1
2      1
3      1
4      1
      ..
195    0
196    0
197    0
198    0
199    0
Name: labels, Length: 200, dtype: int64

##Define a classification model


Given the LogisticRegression Classifier    
fit in the train dataset feature and labels for training

In [118]:
clf = LogisticRegression(random_state=0, solver='lbfgs')
clf.fit(train_df, train_labels)

LogisticRegression(random_state=0)

##Do predication with the trained classifier

use function predict(select a reviews column from test_feature)    
the predicted out should be 1 or 0 (Positive 1 or Negative 0)   



In [119]:
print(clf.predict(test_df.iloc[[1]]))
print(test_labels[1])
print(clf.predict(test_df.iloc[[110]]))
print(test_labels[110])

[0]
1
[0]
0


##Test the accurracy using test dataset

Use clf.score(feature, labels) to calculate the score between 0 to 1    


In [120]:
print(f"The score for complete test reviews is: {clf.score(test_features, test_labels) * 100 } %")

The score for complete test reviews is: 62.5 %


## Q4:

Given the following `dataset1.csv`

Here show the first 5 rows of a population data

| Unnamed: 0 | age | educatn | earnings | hours | kids | married  |
|------------|-----|---------|----------|-------|------|----------|
| 0          | 39  | 12.0    | 77250    | 2940  | 2    | married  |
| 1          | 35  | 12.0    | 12000    | 2040  | 2    | divorced |
| 2          | 33  | 12.0    | 8000     | 693   | 1    | married  |
| 3          | 39  | 10.0    | 15000    | 1904  | 2    | married  |
| 4          | 47  | 9.0     | 6500     | 1683  | 5    | married  |

Given the follow Hypothesis:

Null Hypothesis `HO`:

- Work hours for people with higher earnings == Work hours for people with lower earnings

Alternative Hypothesis `HA`:

- Work hours for people with higher earnings > Work hours for people with lower earnings

Conduct the hypothesis test with sample data using python `scipy.stats`function.

State your result of the hypothesis test.

In [124]:
from scipy import stats

data = pd.read_csv(data_dir_path + 'dataset1.csv')

earnings = data['earnings']
hours = data['hours']

higher_earnings = hours[earnings > earnings.median()]
lower_earnings = hours[earnings <= earnings.median()]

result = stats.ttest_ind(higher_earnings, lower_earnings, alternative='greater')
print(result)

if result.pvalue < 0.05:
    print("Reject the null hypothesis. There is evidence that work hours for people with higher earnings are greater than work hours for people with lower earnings.")
else:
    print("Fail to reject the null hypothesis. There is not enough evidence to conclude that work hours for people with higher earnings are greater than work hours for people with lower earnings.")

Ttest_indResult(statistic=69.1322430856987, pvalue=0.0)
Reject the null hypothesis. There is evidence that work hours for people with higher earnings are greater than work hours for people with lower earnings.


## Q5:
An IOT device collected data in the following format:

1. `dev-144` temperature is 40.5 degree celsius, pressure is 20000.5 pa
2. `dev-146` pressure is 24000.5 pa, temperature is 35.5 degree Celsius
3. `dev-23` temperature is 33.5 degree celsius, pressure is 10400.3 pa
4. `dev-25` pressure is 40040.5 pa, temperature is 24.5 degree Celsius

Use python regular expression to extract the data and store into a Json format.

```
{
  “device”:”dev-144”,
  “temperature”: 40.5,
  “pressure”: 20000.5
}
```

In [125]:
import re
import json

data = [
    "dev-144 temperature is 40.5 degree celsius, pressure is 20000.5 pa",
    "dev-146 pressure is 24000.5 pa, temperature is 35.5 degree Celsius",
    "dev-23 temperature is 33.5 degree celsius, pressure is 10400.3 pa",
    "dev-25 pressure is 40040.5 pa, temperature is 24.5 degree Celsius"
]

device_pattern = r'dev-(\d+)'
temperature_pattern = r'temperature is (\d+\.\d+)'
pressure_pattern = r'pressure is (\d+\.\d+)'

result = []

for item in data:
    device_match = re.search(device_pattern, item)
    temperature_match = re.search(temperature_pattern, item, re.IGNORECASE)
    pressure_match = re.search(pressure_pattern, item, re.IGNORECASE)

    if device_match and temperature_match and pressure_match:
        device = device_match.group(1)
        temperature = float(temperature_match.group(1))
        pressure = float(pressure_match.group(1))

        result.append({
            'device': 'dev-' + device,
            'temperature': temperature,
            'pressure': pressure
        })

json_result = json.dumps(result, indent=4)

print(json_result)


[
    {
        "device": "dev-144",
        "temperature": 40.5,
        "pressure": 20000.5
    },
    {
        "device": "dev-146",
        "temperature": 35.5,
        "pressure": 24000.5
    },
    {
        "device": "dev-23",
        "temperature": 33.5,
        "pressure": 10400.3
    },
    {
        "device": "dev-25",
        "temperature": 24.5,
        "pressure": 40040.5
    }
]
